In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [3]:
# !pip install google-generativeai

  Using cached google_generativeai-0.6.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.4-py3-none-any.whl.metadata (5.6 kB)
  Using cached google_api_core-2.19.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_auth-2.30.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached protobuf-5.27.1-cp310-abi3-win_amd64.whl.metadata (592 bytes)
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------- ---------------------------- 30.7/109.4 kB 1.3 MB/s eta 0:00:01
     ------------- ----------------------- 41.0/109.4 kB 991.0 kB/s eta 0:00:01
     --------------------------------- -- 102.4/109.4 kB 980.4 kB/s eta 0:00:01
     ------------------------------------ 109.4/109.4 kB 910.0 kB/s eta 0:00:00
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached proto_plus-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached go

# OCR

In [2]:
import google.generativeai as genai

d:\FPTU\AskPic-POC\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
sample_file = genai.upload_file(path="images\sample_3.png",
                            display_name="sample test")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

Uploaded file 'sample test' as: https://generativelanguage.googleapis.com/v1beta/files/yk238159u4uz


In [7]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'sample test' as: https://generativelanguage.googleapis.com/v1beta/files/yk238159u4uz


In [8]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

response = model.generate_content(["""Perform OCR on this image""", sample_file])

# Markdown(">" + response.text)

extracted_text = response.text

print(extracted_text)

16. Please give me some **recommendation** to buy suitable books for my ten-year-old girl.
    A. advice        **B. defense**        C. interest          D. question

17. In some countries, guests tend to feel they are not highly **regarded** if the invitation to a dinner party is extended only three or four days before the party date.
    A. admired        B. worshipped       **C. expected**        D. disrespected

18. Parents blame declining reading culture on the **mounting** pressure on the children in schools and tuition classes.
    A. attentive       B. distracted          C. growing          **D. decreasing**

19. Two hours are not long enough for this rest.
    A. hours         B. are             **C. enough**         D. this

20. The kids are really excited with going on holiday.
    A. on             **B. really**          C. with              D. going

21. More open discussions will be conducted online to help us do a good decision.
   A. discussions    B. conducted       

In [54]:
genai.delete_file(sample_file.name)
print(f'Deleted {sample_file.display_name}.')

Deleted sample test.


# QnA

In [19]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import LLMChain
from langchain_core.output_parsers import JsonOutputParser

In [36]:
llm = GoogleGenerativeAI(model="gemini-1.5-pro", api_key=GOOGLE_API_KEY, temperature=0)

In [37]:
parser = JsonOutputParser()

In [38]:
system_message_prompt_template = SystemMessagePromptTemplate.from_template("""
You are an expert in answering questions. Your task is to extract the questions from the following text and answer them.
The given text might include redundant information, you must focus only on the questions.
The questions must be answered in a clear and concise manner.
You MUST ONLY respond a string using this format:
    {{
        "response": [
            {{
                "question": str,
                "answer": str
            }}
            # repeat for each question
        ]
            
    }}
""")
human_message_prompt_template = HumanMessagePromptTemplate.from_template("""
This is the text you need to extract questions from: {text}
""")

In [39]:
prompt_template = ChatPromptTemplate.from_messages(
    [
    system_message_prompt_template, human_message_prompt_template
    ]
)

In [50]:
chain = prompt_template | llm | parser

In [51]:
response = chain.invoke({"text": extracted_text})

In [52]:
answers = response["response"]

In [53]:
answers

[{'question': 'Mark the letter A, B, C or D on your answer sheet to indicate the word whose underlined part differs from the other three in pronunciation in the following question.\nA. m_u_lticultural B. _u_rban C. ad_u_lthood D. _s_culpture',
  'answer': 'The answer is **D. _s_culpture**. The others are pronounced with the sound /ʌ/.'},
 {'question': 'Mark the letter A, B, C or D on your answer sheet to indicate the word whose underlined part differs from the other three in pronunciation in the following question.\nA. fascinat_i_ng B. sc_i_ssors C. consc_i_ous D. cultural',
  'answer': 'The answer is **B. sc_i_ssors**. The others are pronounced with the sound /ə/.'},
 {'question': 'Mark the letter A, B, C or D on your answer sheet to indicate the word that differs from the other three in the position of primary stress in the following question.\nA. awareness B. profession C. picturesque D. occasion',
  'answer': 'The answer is **C. picturesque**. The stress is on the first syllable fo